# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [16]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [29]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [19]:
# set up environment

load_dotenv(override=True)

api_key = os.getenv('OPENROUTER_API_KEY')
base_url = os.getenv('OPENROUTER_BASE_URL')

api_key_ollama = os.getenv('OLLAMA_API_KEY')
base_url_ollama = os.getenv('OLLAMA_BASE_URL')

if api_key and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

    

API key looks good so far


In [20]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [25]:
# Get gpt-4o-mini to answer, with streaming

openai = OpenAI(base_url=base_url, api_key=api_key)
chunks = openai.chat.completions.create(
    model=MODEL_GPT, 
    messages=[
        {"role": "system", "content": "You are a technical assistant that can answer questions."},
        {"role": "user", "content": question},
    ],
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in chunks:
    response += chunk.choices[0].delta.content
    update_display(Markdown(response), display_id=display_handle.display_id)




The code snippet you provided uses a combination of a set comprehension and the `yield from` statement in Python. Here's a breakdown of what it does:

1. **Set Comprehension**: 
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set of unique authors extracted from a collection of `books`.
   - It iterates through each `book` in the variable `books`, which is expected to be a collection (like a list or a dictionary) of book objects (typically dictionaries).
   - The `book.get("author")` accesses the value associated with the key `"author"` in each `book` dictionary.
   - The `if book.get("author")` condition checks whether the author exists (i.e., it is not `None` or an empty string). If it exists, the author is added to the set.

2. **`yield from`**:
   - The `yield from` statement is used in Python generator functions. It is a way to yield all values from an iterable (in this case, the set created by the comprehension).
   - Using `yield from` allows the generator to yield each author in the set one by one. This means the function containing this line of code will produce a generator that emits each unique author found in the `books` collection.

### Summary
In summary, this line of code:
- Collects all unique authors from a list of `books`, ignoring any books without an author.
- It returns these authors as a generator, yielding them one at a time.

This approach is useful when you want to efficiently retrieve and process unique authors from a potentially large dataset without needing to store them all in memory at once.

In [30]:
# Get Llama 3.2 to answer
openai = OpenAI(base_url=base_url_ollama, api_key=api_key_ollama)
response = openai.chat.completions.create(
    model=MODEL_LLAMA, 
    messages=[
        {"role": "system", "content": "You are a technical assistant that can answer questions."},
        {"role": "user", "content": question},
    ]
)

response.choices[0].message.content

'This line of code uses the `yield` keyword with a generator expression to process a list of book objects.\n\nHere\'s a breakdown:\n\n- `yield`: This keyword is used to create a function that can produce more than one output value on multiple paths. It returns values from the execution point and pauses the execution until the iteration finishes.\n\n- `{book.get("author") for book in books}`: This part creates an infinite loop using a generator expression, which yields each item in `books` at regular intervals. The outer `{ ... }` is a list comprehension that iterates over all items in `books`. \n\n    - `book.get("author"):` Retrieves the value of "author" from each book object. If a book does not have an "author", this will raise an error because you\'re trying to access non-existent attribute.\n\n- `yield from`: This part takes all the results yielded by the previous expression and yields them separately, without creating a new list.\n\nTherefore, when executed, this code processes a